# Simple retirement model

The retirement model from the "Dynamic programming" course is implemented

## Model equations

The model's **bellman equation** is given by

$$ \begin{eqnarray}
V_t \left(m_t, z_t, \varepsilon_t^0, \varepsilon_t^1 \right) &=&
\max_{z_{t+1} \in \mathcal{Z}(z_t)} \lbrace v_t(m_t | z_{t+1}) + \sigma_{\varepsilon} \varepsilon_t^{z_{t+1}} \rbrace \\
v_t(m_t | z_{t+1}) &=& 
\max_{c_t} \frac{c_t^{1 - \rho}}{1 - \rho} - \alpha \mathbb{1}_{\{z_{t+1} = 0\}} + \beta \mathbb{E}_t \left[V_{t+1}
\left(\cdot_{t+1} \right) \right] \\
&\text{s.t.}& \\
\mathcal{Z}(z_t) &=& 
\begin{cases}
\{0,1\} \quad & \text{if } z_t = 0 \\
\{1\} \quad & \text{if } z_t = 1 
\end{cases} \\
m_{t+1} &=& R(m_t - c_t) + W \xi_{t+1} \mathbb{1}_{\{z_{t+1} = 0\}} \\
c_t &\leq& m_t \\
\ln \left(\xi_{t+1} \right) &\sim& \mathcal{N} \left(-0.5 \sigma_{\xi}^2, \sigma_{\xi}^2 \right) \\
\varepsilon_{t+1}^0, \varepsilon_{t+1}^1 &\sim& \text{Extreme Value Type 1}
\end{eqnarray}$$

# Test

## Numba

In [1]:
from consav import runtools
runtools.write_numba_config(disable=0,threads=8)

## Setup

In [2]:
%matplotlib inline

# reload module each time cell is run
%load_ext autoreload
%autoreload 2

In [4]:
# load the BufferStockModel module
from RetirementModel import RetirementModelClass

## Example

In [5]:
# a. setup (calling the __init__ method) 
model = RetirementModelClass(name='baseline',solmethod='egm')

# b. print
print(model)

# c. solve
model.solve()

Modelclass: RetirementModelClass
Parameters:
 T = 20
 rho = 2
 beta = 0.96
 alpha = 0.75
 sigma_eta = 0.0
 W = 1
 sigma_xi = 0.2
 R = 1.04
 a_max = 10
 a_phi = 1.1
 Nxi = 8
 Na = 150
 grid_a = [array of doubles]
 xi = [array of doubles]
 xi_w = [array of doubles]
 a_work = [array of doubles]
 xi_work = [array of doubles]
 tol = 1e-08
 do_print = True
 do_simple_w = False
 cppthreads = 1
 simT = 20
 simN = 1000
 sim_seed = 1998

Iteration 18 : 0.03493547439575195
envelope

Iteration 17 : 0.047913312911987305
envelope

Iteration 16 : 0.0479276180267334
envelope

Iteration 15 : 0.06038832664489746
envelope

Iteration 14 : 0.0818486213684082
envelope

Iteration 13 : 0.052906036376953125
envelope

Iteration 12 : 0.06137990951538086
Iteration 11 : 0.03843379020690918
Iteration 10 : 0.04791140556335449
Iteration 9 : 0.08484220504760742
Iteration 8 : 0.08084416389465332
Iteration 7 : 0.0733642578125
Iteration 6 : 0.08434224128723145
Iteration 5 : 0.05938863754272461
Iteration 4 : 0.06388163566

## Checksums - comparing with Matlab code

In [6]:
# Retired
sum_ret = np.sum(model.sol.c[:,:,0])

# Working
sum_work = np.sum(model.sol.c[:,:,1])

In [7]:
sum_ret/1000

2.9586892051942333

In [8]:
sum_work/1000

4.271682106738003

When using linear_interp_vec_mon_rep something odd happens to the check_sum sum_work (3.528 vs 4.271)

The model is comparable with the version in matlab.
I have not added 10 points on the constraint, when doing egm.

When the upper envelope is invoked the results differ a bit more

# Timing

In [9]:
import time

In [10]:
model = RetirementModelClass(name='baseline',solmethod='egm')
tic = time.time()
model.solve()
toc = time.time()

print('with numba it takes:', toc-tic)

Iteration 18 : 0.06487894058227539
envelope

Iteration 17 : 0.06537723541259766
envelope

Iteration 16 : 0.04491758346557617
envelope

Iteration 15 : 0.045914411544799805
envelope

Iteration 14 : 0.0444183349609375
envelope

Iteration 13 : 0.05489635467529297
envelope

Iteration 12 : 0.04790806770324707
Iteration 11 : 0.03443646430969238
Iteration 10 : 0.04791140556335449
Iteration 9 : 0.035431861877441406
Iteration 8 : 0.04142355918884277
Iteration 7 : 0.06338119506835938
Iteration 6 : 0.058391571044921875
Iteration 5 : 0.042920589447021484
Iteration 4 : 0.03942751884460449
Iteration 3 : 0.06138467788696289
Iteration 2 : 0.05140542984008789
Iteration 1 : 0.06387925148010254
Iteration 0 : 0.06038951873779297
with numba it takes: 1.0011396408081055


In [11]:
import cProfile
cProfile.run('model.solve()')

Iteration 18 : 0.05439877510070801
envelope

Iteration 17 : 0.0783543586730957
envelope

Iteration 16 : 0.057393550872802734
envelope

Iteration 15 : 0.06338286399841309
envelope

Iteration 14 : 0.05889272689819336
envelope

Iteration 13 : 0.0494084358215332
envelope

Iteration 12 : 0.05190300941467285
Iteration 11 : 0.03892874717712402
Iteration 10 : 0.0444183349609375
Iteration 9 : 0.05739283561706543
Iteration 8 : 0.06587767601013184
Iteration 7 : 0.0563967227935791
Iteration 6 : 0.0444183349609375
Iteration 5 : 0.05439925193786621
Iteration 4 : 0.04292011260986328
Iteration 3 : 0.04192185401916504
Iteration 2 : 0.06538748741149902
Iteration 1 : 0.0683736801147461
Iteration 0 : 0.042920827865600586
         295925 function calls in 1.067 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.067    1.067 <string>:1(<module>)
        1    0.001    0.001    0.001    0.001 RetirementModel.py:215(_s